In [ ]:
# Numpy imports
import numpy as np
from numpy import arcsinh

# Matplotlib imports
import matplotlib.pyplot as plt

# Non-astro imports
from PIL import Image
from PIL import ImageFilter

# Astropy imports
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.io.fits import HDUList
from astropy.nddata import Cutout2D
from astropy import units as u
from astropy.visualization import make_lupton_rgb
from astropy.wcs import WCS

# Other astro imports
import aplpy
from aplpy.rgb import make_rgb_cube
from reproject import reproject_interp

In [ ]:
%matplotlib inline

In [ ]:
%ls -l

In [ ]:
fits_files = ['frame-i-000756-2-0427.fits', 'frame-r-000756-2-0427.fits', 'frame-g-000756-2-0427.fits']

In [ ]:
output = 'rgb-000756-2-0427.fits'

In [ ]:
make_rgb_cube(fits_files, output)

In [ ]:
# Make an RGB image
aplpy.make_rgb_image('rgb-000756-2-0427.fits', 'rgb-000756-2-0427.png', stretch_r='arcsinh', stretch_g='arcsinh', stretch_b='arcsinh')

# Plot the RGB image using the 2d image to indicate the projection
f = aplpy.FITSFigure('rgb-000756-2-0427_2d.fits')
f.show_rgb('rgb-000756-2-0427.png')

In [ ]:
fitsimage = fits.open('rgb-000756-2-0427.fits')
hdu = fitsimage[0]
header = hdu.header
data = hdu.data
wcs = WCS(header, naxis=2)

In [ ]:
data[np.isnan(data)] = -100

In [ ]:
n, x_shape, y_shape = data.shape

In [ ]:
def scale_rgb(data, sigma=6, vmin=0.0, vmax=255.0, gains=[0.9,1.1,1.8], gamma=0.1):
    slope = 255.0 /np.sqrt(arcsinh((vmax - vmin)/sigma))
    R_IDX = 0
    G_IDX = 1
    B_IDX = 2

    r = data[R_IDX].copy()
    g = data[G_IDX].copy()
    b = data[B_IDX].copy()

    i = (r + g + b) / 3
    min_mask = (i <= vmin)
    
    scale = slope * np.sqrt(arcsinh((i - vmin)/sigma)) / i
    scale[min_mask] = 0

    r = (r * scale).astype(int)
    g = (g * scale).astype(int)
    b = (b * scale).astype(int)
    
    r = (r * gains[R_IDX]).astype(int)
    g = (g * gains[G_IDX]).astype(int)
    b = (b * gains[B_IDX]).astype(int)
    
    r = (r + gamma * (r - g)).astype(int)
    b = (b + gamma * (b - g)).astype(int)

    r[r < 0] = 0
    g[g < 0] = 0
    b[b < 0] = 0

    r[r > 255] = 255
    g[g > 255] = 255
    b[b > 255] = 255
    
    result = np.empty((r.shape[0],r.shape[1],3), dtype=np.uint8)
    result[:,:,0] = r
    result[:,:,1] = g
    result[:,:,2] = b

    return result

In [ ]:
r = data[0]
g = data[1]
b = data[2]

In [ ]:
rgb_default = make_lupton_rgb(r, g, b, filename='rgb-000756-2-0427.png', Q=8, stretch=0.4)
plt.figure(figsize=(20,40))
plt.imshow(rgb_default, origin='lower')

In [ ]:
scaled_rgb = scale_rgb(data, sigma=6)
print(scaled_rgb.max())
plt.figure(figsize=(20,40))
plt.imshow(scaled_rgb, origin='lower')

In [ ]:
def world2pix(ra, dec, wcs):
    coords = np.array([[ra, dec]])
    location = wcs.wcs_world2pix(coords, 0, ra_dec_order=True)
    return location[0][0], location[0][1]

def cutout(data, wcs, ra, dec, x_size=100, y_size=100):
    x_centre, y_centre = world2pix(ra, dec, wcs)
    x_top = int(round(x_centre) - x_size/2)
    y_top = int(round(y_centre) - y_size/2)
    x_bottom = x_top + x_size
    y_bottom = y_top + y_size
    
    return data[y_top:y_bottom, x_top:x_bottom]

In [ ]:
r_cutout = cutout(r, wcs, 179.689293428, -0.454379058, x_size=224, y_size=224)
g_cutout = cutout(g, wcs, 179.689293428, -0.454379058, x_size=224, y_size=224)
b_cutout = cutout(b, wcs, 179.689293428, -0.454379058, x_size=224, y_size=224)

r_cutout[r_cutout < 0] = 0
g_cutout[g_cutout < 0] = 0
b_cutout[b_cutout < 0] = 0

vmax = max(r.max(), g.max(), b.max())

In [ ]:
rgb_cutout = make_lupton_rgb(r_cutout, g_cutout, b_cutout, filename='rgb-cutout-000756-2-0427.png', Q=8, stretch=0.5)
plt.figure(figsize=(10,10))
plt.imshow(rgb_cutout, origin='lower')

In [ ]:
kernel = Gaussian2DKernel(x_stddev=0.3)
rgb_convoled = rgb_cutout.copy()

rgb_convoled[:,:,0] = convolve(rgb_cutout[:,:,0], kernel)
rgb_convoled[:,:,1] = convolve(rgb_cutout[:,:,1], kernel)
rgb_convoled[:,:,2] = convolve(rgb_cutout[:,:,2], kernel)

plt.figure(figsize=(10,10))
plt.imshow(rgb_convoled, origin='lower')

In [ ]:
%matplotlib inline

def plot_rgb(image_r, image_g, image_b, image,
             rtitle='', gtitle='', btitle=''):
    fig = plt.figure(figsize=(16, 4))
    ax1 = fig.add_subplot(142)
    ax2 = fig.add_subplot(143)
    ax3 = fig.add_subplot(144)
    ax4 = fig.add_subplot(141)

    ax1.imshow(image_r, cmap='Reds')
    ax1.axes.get_xaxis().set_visible(False)
    ax1.axes.get_yaxis().set_visible(False)
    ax1.set_title(rtitle)

    ax2.imshow(image_g, cmap='Greens')
    ax2.axes.get_xaxis().set_visible(False)
    ax2.axes.get_yaxis().set_visible(False)
    ax2.set_title(gtitle)

    ax3.imshow(image_b, cmap='Blues')
    ax3.axes.get_xaxis().set_visible(False)
    ax3.axes.get_yaxis().set_visible(False)
    ax3.set_title(btitle)

    ax4.imshow(image)
    ax4.axes.get_xaxis().set_visible(False)
    ax4.axes.get_yaxis().set_visible(False)

In [ ]:
position = SkyCoord(ra=179.689293428*u.deg, dec=-0.454379058*u.deg, frame='icrs')
cutout_size = u.Quantity((25,25), u.arcsec)
r_cutout = Cutout2D(r,position=position,size=cutout_size, wcs=wcs)
g_cutout = Cutout2D(g,position=position,size=cutout_size, wcs=wcs)
b_cutout = Cutout2D(b,position=position,size=cutout_size, wcs=wcs)

In [ ]:
# r_cutout_data = convolve(r_cutout_1.data, kernel)
# g_cutout_data = convolve(g_cutout_1.data, kernel)
# b_cutout_data = convolve(b_cutout_1.data, kernel)

# r_cutout_data = r_cutout_1.data
# g_cutout_data = g_cutout_1.data
# b_cutout_data = b_cutout_1.data

# r_cutout_data[r_cutout_data < 0] = 0
# g_cutout_data[g_cutout_data < 0] = 0
# b_cutout_data[b_cutout_data < 0] = 0

rgb = np.empty((3, r_cutout.shape[0], r_cutout.shape[1]))
print(rgb.shape)
rgb[0] = r_cutout.data
rgb[1] = g_cutout.data
rgb[2] = b_cutout.data

rgb[rgb < 0] = 0
#rgb /= rgb.max()
#rgb /= rgb/vmax
#print(rgb.max())
#print((rgb/vmax).max())

modified_cutout = scale_rgb(rgb, sigma=2)

#modified_cutout = make_lupton_rgb(rgb[0]/vmax,rgb[1]/vmax,rgb[2]/vmax, filename='rgb-cutout-000756-2-0427.png', Q=0.00001, stretch=0.2)

# rgb_cutout = make_lupton_rgb(r_cutout_data/vmax, g_cutout_data/vmax, b_cutout_data/vmax, filename='rgb-cutout-000756-2-0427.png', Q=10, stretch=0.2)

# modified_cutout[:,:,0] = convolve(modified_cutout[:,:,0], kernel)
# modified_cutout[:,:,1] = convolve(modified_cutout[:,:,1], kernel)
# modified_cutout[:,:,2] = convolve(modified_cutout[:,:,2], kernel)

plt.figure(figsize=(5,5))
plt.imshow(modified_cutout, origin='lower')

In [ ]:
rescaled_image = Image.fromarray(modified_cutout).resize((512,512), Image.BILINEAR).transpose(Image.FLIP_TOP_BOTTOM)
rescaled_image = rescaled_image.filter(ImageFilter.GaussianBlur(radius=4))
plt.figure(figsize=(10,10))
plt.imshow(rescaled_image)

In [ ]:
rescaled_image.save('rescaled_cutout.jpeg')

In [ ]:
vmax

In [ ]:
r > 0